# Hackathon: From Raw Data to ML-Ready Dataset
## Insight-Driven EDA and End-to-End Feature Engineering on Airbnb Data Using pandas and Plotly

### What is a Hackathon?

A hackathon is a fast-paced, collaborative event where participants use data and technology to solve a real problem end-to-end.  
In this hackathon, you will work with a **real-world Airbnb dataset** and complete two interconnected goals:

- Produce a **high-quality exploratory data analysis (EDA)** using `pandas` and `plotly`, extracting meaningful insights, trends, and signals from the data.  
- Design and deliver a **clean, feature-rich, ML-ready dataset** that will serve as the foundation for a follow-up hackathon focused on building and evaluating machine learning models.

Your task is to **get the most out of the data**: uncover structure and patterns through EDA, and engineer informative features (numerical, categorical, temporal, textual (TF–IDF), and optionally image-based) to maximize the predictive power of the final dataset.

<div class="alert alert-success">
<b>About the Dataset</b>

<u>Context</u>

The data comes from <a href="https://insideairbnb.com/get-the-data/">Inside Airbnb</a>, an open project that publishes detailed, regularly updated datasets for cities around the world.  
Each city provides three main CSV files:

- <b>listings.csv</b> — property characteristics, host profiles, descriptions, amenities, etc.  
- <b>calendar.csv</b> — daily availability and pricing information for each listing.  
- <b>reviews.csv</b> — guest feedback and textual reviews.

These datasets offer a rich view of the short-term rental market, including availability patterns, pricing behavior, host attributes, and guest sentiment.  

<u>Inspiration</u>

Your ultimate objective is to create a dataset suitable for training a machine learning model that predicts whether a specific Airbnb listing will be <b>available on a given date</b>, using property attributes, review information, and host characteristics.
</div>

<div class="alert alert-info">
<b>Task</b>

Using one city of your choice from Inside Airbnb, create an end-to-end pipeline that:

1. Loads and explores the raw data (EDA).  
2. Engineers features (numerical, categorical, temporal, textual TF–IDF, etc.).  
3. Builds a unified ML-ready dataset.  

Please remember to add comments explaining your decisions. Comments help us understand your thought process and ensure accurate evaluation of your work. This assignment requires code-based solutions—**manually calculated or hard-coded results will not be accepted**. Thoughtful comments and visualizations are encouraged and will be highly valued.

- Write your solution directly in this notebook, modifying it as needed.
- Once completed, submit the notebook in **.ipynb** format via Moodle.
    
<b>Collaboration Requirement: Git & GitHub</b>

You must collaborate with your team using a **shared GitHub repository**.  
Your use of Git is part of the evaluation. We will specifically look at:

- Commit quality (clear messages, meaningful steps).  
- Balanced participation across team members.  
- Use of branches.  
- Ability to resolve merge conflicts appropriately.  
- A clean, readable project history that reflects real collaboration.

Good Git practice is **part of your grade**, not optional.
</div>
<div class="alert alert-danger">
    You are free to add as many cells as you wish as long as you leave untouched the first one.
</div>

<div class="alert alert-warning">

<b>Hints</b>

- Text columns often carry substantial predictive power, use text-vectorization methods to extract meaningful features.  
- Make sure all columns use appropriate data types (categorical, numeric, datetime, boolean). Correct dtypes help prevent subtle bugs and improve performance.  
- Feel free to enrich the dataset with any additional information you consider useful: engineered features, external data, derived temporal features, etc.  
- If the dataset is too large for your computer, use <code>.sample()</code> to work with a subset while preserving the logic of your pipeline.  
- Plotly offers a wide variety of powerful visualizations, experiment creatively, but always begin with a clear analytical question: *What insight am I trying to uncover with this plot?*

</div>




<div class="alert alert-danger">
<b>Submission Deadline:</b> Wednesday, December 3rd, 12:00

Start with a simple, working pipeline.  
Do not over-complicate your code too much. Start with a simple working solution and refine it if you have time.
</div>

<div class="alert alert-danger">
    
You may add as many cells as you want, but the **first cell must remain exactly as provided**. Do not edit, move, or delete it under any circumstances.
</div>


In [1]:
# LEAVE BLANK

### Team Information

Fill in the information below.  
All fields are **mandatory**.

- **GitHub Repository URL**: Paste the link to the team repo you will use for collaboration.
- **Team Members**: List all student names (and emails or IDs if required).

Do not modify the section title.  
Do not remove this cell.


In [2]:
# === Team Information (Mandatory) ===
# Fill in the fields below.

GITHUB_REPO = "https://github.com/seanhoet65-source/hackathon_python"
TEAM_MEMBERS = ["Pau Gratacós Fusté", "Sean Hoet", "Florian Nix", "Caroline Wheeler", "Riwad Irshied"]

GITHUB_REPO, TEAM_MEMBERS


('https://github.com/seanhoet65-source/hackathon_python',
 ['Pau Gratacós Fusté',
  'Sean Hoet',
  'Florian Nix',
  'Caroline Wheeler',
  'Riwad Irshied'])

In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# ===== 1. LOAD THE RAW DATA =====
print("="*60)
print("STEP 1: LOADING RAW DATA")
print("="*60)

# Local paths
listings_path = "listings.csv.gz"
calendar_path = "calendar.csv.gz"
reviews_path = "reviews.csv.gz"

# Load datasets
listings = pd.read_csv(listings_path)
calendar = pd.read_csv(calendar_path)
reviews = pd.read_csv(reviews_path)

print(f"✓ Listings: {listings.shape[0]:,} rows × {listings.shape[1]} columns")
print(f"✓ Calendar: {calendar.shape[0]:,} rows × {calendar.shape[1]} columns")
print(f"✓ Reviews: {reviews.shape[0]:,} rows × {reviews.shape[1]} columns")

# ===== 2. CLEAN LISTINGS DATA =====
print("\n" + "="*60)
print("STEP 2: CLEANING LISTINGS DATA")
print("="*60)

original_listings = listings.shape[0]

# Convert date columns
if "last_scraped" in listings.columns:
    listings["last_scraped"] = pd.to_datetime(listings["last_scraped"], errors="coerce")
if "host_since" in listings.columns:
    listings["host_since"] = pd.to_datetime(listings["host_since"], errors="coerce")

# Clean price column from listings (this will be our main price signal)
if "price" in listings.columns:
    listings["price"] = listings["price"].replace(r"[\$,]", "", regex=True)
    listings["price"] = pd.to_numeric(listings["price"], errors="coerce")

# Convert boolean-like columns
bool_mapping = {"t": True, "f": False, True: True, False: False}
bool_cols = [
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
    "has_availability",
    "instant_bookable",
]
for col in bool_cols:
    if col in listings.columns:
        listings[col] = listings[col].map(bool_mapping)

# Convert percentage-like columns
numeric_cols = ["host_response_rate", "host_acceptance_rate"]
for col in numeric_cols:
    if col in listings.columns and listings[col].dtype == "object":
        listings[col] = listings[col].str.rstrip("%").astype(float) / 100

# Remove listings with missing or invalid key info
listings = listings.dropna(subset=["id", "price"])
listings = listings[listings["price"] > 0]

print(f"✓ Removed {original_listings - listings.shape[0]:,} listings with missing/invalid data")
print(f"✓ Clean listings: {listings.shape[0]:,} rows")

# ===== 3. CLEAN CALENDAR DATA =====
print("\n" + "="*60)
print("STEP 3: CLEANING CALENDAR DATA")
print("="*60)

original_calendar = calendar.shape[0]

# Convert date column
calendar["date"] = pd.to_datetime(calendar["date"], errors="coerce")

# Try to clean price columns if they exist (may be all NaN in this dataset)
for col in ["price", "adjusted_price"]:
    if col in calendar.columns:
        calendar[col] = calendar[col].replace(r"[\$,]", "", regex=True)
        calendar[col] = pd.to_numeric(calendar[col], errors="coerce")

# Convert available to boolean (target)
if "available" in calendar.columns:
    calendar["available"] = calendar["available"].map({"t": True, "f": False})

# Remove rows with missing critical data
calendar = calendar.dropna(subset=["listing_id", "date", "available"])

print(f"✓ Removed {original_calendar - calendar.shape[0]:,} calendar entries with missing data")
print(f"✓ Clean calendar: {calendar.shape[0]:,} rows")
print(f"✓ Date range: {calendar['date'].min()} to {calendar['date'].max()}")

# ===== 4. CLEAN REVIEWS DATA =====
print("\n" + "="*60)
print("STEP 4: CLEANING REVIEWS DATA")
print("="*60)

original_reviews = reviews.shape[0]

# Convert date column
reviews["date"] = pd.to_datetime(reviews["date"], errors="coerce")

# Remove rows with missing critical data
reviews = reviews.dropna(subset=["listing_id", "date"])

# Remove reviews with no/very short text
if "comments" in reviews.columns:
    reviews = reviews.dropna(subset=["comments"])
    reviews = reviews[reviews["comments"].str.len() > 10]

print(f"✓ Removed {original_reviews - reviews.shape[0]:,} reviews with missing data")
print(f"✓ Clean reviews: {reviews.shape[0]:,} rows")

# ===== 5. AGGREGATE REVIEW FEATURES =====
print("\n" + "="*60)
print("STEP 5: CREATING REVIEW FEATURES")
print("="*60)

review_features = (
    reviews.groupby("listing_id")
    .agg(
        {
            "id": "count",          # total number of reviews
            "date": ["min", "max"], # first and last review dates
        }
    )
    .reset_index()
)

# Flatten column names
review_features.columns = [
    "listing_id",
    "review_count",
    "first_review_date",
    "last_review_date",
]

current_date = pd.Timestamp.now()
review_features["days_since_first_review"] = (
    current_date - review_features["first_review_date"]
).dt.days
review_features["days_since_last_review"] = (
    current_date - review_features["last_review_date"]
).dt.days
review_features["review_period_days"] = (
    review_features["last_review_date"] - review_features["first_review_date"]
).dt.days

# Review rate (reviews per month)
review_features["reviews_per_month"] = (
    review_features["review_count"] / (review_features["review_period_days"] / 30)
)
review_features["reviews_per_month"] = review_features["reviews_per_month"].fillna(0)

print(f"✓ Review features created for {review_features.shape[0]:,} listings")
print("\nReview features preview:")
print(review_features.head())

# ===== 6. MERGE DATASETS =====
print("\n" + "="*60)
print("STEP 6: MERGING DATASETS")
print("="*60)

# Merge calendar with listings: base grain = (listing, date)
df = calendar.merge(
    listings,
    left_on="listing_id",
    right_on="id",
    how="inner",
    suffixes=("_cal", "_listing"),
)
print(f"✓ Calendar + Listings: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Add review features
df = df.merge(
    review_features,
    on="listing_id",
    how="left",  # keep all listing-date rows even if no reviews
)
print(f"✓ Added review features: {df.shape[0]:,} rows × {df.shape[1]} columns")

# ===== 6b. UNIFY PRICE + BASIC CLEANUP =====

# Create a single 'price' column:
# Prefer calendar price if it exists and is non-null, otherwise use listing price.
price_cal_col = "price_cal" if "price_cal" in df.columns else None
price_listing_col = "price_listing" if "price_listing" in df.columns else None

if price_cal_col and price_listing_col:
    df["price"] = df[price_cal_col].fillna(df[price_listing_col])
elif price_listing_col:
    df["price"] = df[price_listing_col]
elif price_cal_col:
    df["price"] = df[price_cal_col]
else:
    print("⚠ No price columns found after merge!")

# Drop clearly useless / fully missing columns if they exist
cols_to_drop = [
    "price_cal",
    "adjusted_price",                 # all NaN in this scrape
    "license",
    "neighbourhood_group_cleansed",
    "calendar_updated",
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# Fill missing review-derived fields (no reviews → zeros)
review_cols = [
    "review_count",
    "days_since_first_review",
    "days_since_last_review",
    "review_period_days",
    "reviews_per_month",
]
for col in review_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# ===== 7. FINAL DATA QUALITY CHECK =====
print("\n" + "="*60)
print("STEP 7: FINAL DATA QUALITY CHECK")
print("="*60)

print(f"\nFinal dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

print("\nTarget variable (available) distribution:")
print(df["available"].value_counts())

print("\nMissing values in key columns:")
key_cols = ["available", "price", "listing_id", "date"]
existing_key_cols = [c for c in key_cols if c in df.columns]
missing_summary = df[existing_key_cols].isna().sum()
print(missing_summary)

print("\nTop 10 columns with missing values:")
top_missing = df.isna().sum().sort_values(ascending=False).head(10)
print(top_missing)

print("\n" + "="*60)
print("✅ DATA LOADING, CLEANING & MERGING COMPLETE!")
print("="*60)
print(f"Ready for EDA with {df.shape[0]:,} observations")

print("\nSample of merged dataset:")
print(df.head())


STEP 1: LOADING RAW DATA
✓ Listings: 2,654 rows × 79 columns
✓ Calendar: 968,710 rows × 7 columns
✓ Reviews: 122,622 rows × 6 columns

STEP 2: CLEANING LISTINGS DATA
✓ Removed 357 listings with missing/invalid data
✓ Clean listings: 2,297 rows

STEP 3: CLEANING CALENDAR DATA
✓ Removed 0 calendar entries with missing data
✓ Clean calendar: 968,710 rows
✓ Date range: 2025-06-25 00:00:00 to 2026-06-29 00:00:00

STEP 4: CLEANING REVIEWS DATA
✓ Removed 3,155 reviews with missing data
✓ Clean reviews: 119,467 rows

STEP 5: CREATING REVIEW FEATURES
✓ Review features created for 2,222 listings

Review features preview:
   listing_id  review_count first_review_date last_review_date  \
0       50904             3        2015-05-06       2022-05-15   
1      345959           128        2012-05-12       2025-06-19   
2      366252           159        2012-03-29       2024-09-26   
3      603545            45        2012-12-16       2024-11-08   
4      772842            81        2013-01-04      

In [ ]:
df["price"].describe()
'''
count      838,405   ← ✔️ every row has a price
mean          282.7   ← ✔️ reasonable for Barcelona
std          2579     ← ⚠️ huge → indicates some extreme outliers
min            13     ← ✔️ normal low-end price
25%            63     ← ✔️ budget listings
50% (=median)  88     ← ✔️ central tendency
75%           130     ← ✔️ typical Airbnb
max         50,000    ← ⚠️ extreme outliers

'''


count    838405.000000
mean        282.703091
std        2579.411527
min          13.000000
25%          63.000000
50%          88.000000
75%         130.000000
max       50000.000000
Name: price, dtype: float64